# Lista de exercícios 2

## Questão 1

In [1]:
import json
import numpy as np
from math import sin, cos, sqrt, atan2, radians

In [2]:
def distance_between_coords(coord1, coord2):
    """
    Function found in 
    https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude/43211266#43211266
    """
    R = 6373.0
    lat1 = radians(coord1[1])
    lon1 = radians(coord1[0])
    lat2 = radians(coord2[1])
    lon2 = radians(coord2[0])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

In [3]:
with open ('coordinates.txt', 'r', encoding = 'UTF-8') as f:
    info = f.readlines()
lines = [txt.replace('\n', '') for txt in info if txt != '\n']
i = 0
coordinates = {}
while i < len(lines):
    key = lines[i].split('-')[0].strip()
    first_coord = lines[i+1].find('º')
    second_coord = lines[i+1].find('\'')
    third_coord = lines[i+1].find('"')
    lat = -1 * (int(lines[i+1][:first_coord]) + 
                int(lines[i+1][first_coord+1:second_coord])/60 + 
                int(lines[i+1][second_coord+1:third_coord])/3600)
    
    first_coord = lines[i+2].find('º')
    second_coord = lines[i+2].find('\'')
    third_coord = lines[i+2].find('"')
    
    lon = -1 * (int(lines[i+2][:first_coord]) + 
                int(lines[i+2][first_coord+1:second_coord])/60 + 
                int(lines[i+2][second_coord+1:third_coord])/3600)
    coordinates[key] = [lon, lat]
    i += 3

cities = list(coordinates.keys())
cities.remove('Macapá')
cities.remove('Palmas')
cities = cities[0:20]
cities_dict = dict([(cities[i], i) for i in range(len(cities))])

In [4]:
dists_euclid = np.zeros((len(cities), len(cities)))
for i in range(20):
    for j in range(20):
        dists_euclid[i, j] = distance_between_coords(coordinates[cities[i]], coordinates[cities[j]])

In [5]:
dists_road = np.zeros((len(cities), len(cities)))
with open("distancias.json", 'r', encoding='utf-8') as f:
    temp = json.load(f)
for i in range(20):
    for j in range(20):
        city1 = cities[i]
        city2 = cities[j]
        if city1+":"+city2 in temp.keys():
            dist = temp[city1+":"+city2]
        else:
            dist = temp[city2+":"+city1]
        dists_road[i, j] = dist

In [36]:
class Node():
    """A node class for A* Pathfinding"""

    def __init__(self, name = None, parent=None):
        self.parent = parent
        self.name = name
        self.g = 0
        self.h = 0
        self.f = 0
    
    def __eq__(self, other):
        return self.name == other.name

    
class A_star():
    def __init__(self, graph_dist, heuristic_dist, names):
        self.graph_dist = graph_dist
        self.heuristic_dist = heuristic_dist
        self.names = names
        self.names_dict = dict([(names[i], i) for i in range(len(names))])
        
    def run(self, start, end):
        print("A* algorithm - road between cities")
        print(f"start = {start}, end = {end}")
        
        openList = []
        closedList = []
        
        startNode = Node(start, None)
        startNode.g, startNode.h, startNode.f = 0, 0, 0
        
        openList.append(startNode)
        while len(openList) > 0:
            
            nodeBest = openList[0]
            valueBest = nodeBest.f
            for node in openList:
                if node.f < nodeBest.f:
                    nodeBest = node
            
            print(f"Current best = {nodeBest.name}, f = {nodeBest.f}")
            
            openList.remove(nodeBest)
            closedList.append(nodeBest)
            
            if nodeBest.name == end:
                print(f"Reached {end}")
                print("Path (in reverse order):")
                nodeP = nodeBest
                while nodeP.name != startNode.name:
                    print(nodeP.name)
                    nodeP = nodeP.parent
                print(startNode.name)
                return 
            
            newPath = [Node(destination, nodeBest) for destination in self.names]
            for path in newPath:
                
                if path in closedList:
                    continue
                        
                path.g = nodeBest.g + self.graph_dist[self.names_dict[path.name],
                                                     self.names_dict[nodeBest.name]]
                
                path.h = self.heuristic_dist[self.names_dict[path.name],
                                            self.names_dict[end]]
                
                path.f = path.g + path.h
                
                for openPath in openList:
                    if path == openPath and path.g > openPath.g:
                        continue
                        
                openList.append(path)

In [37]:
A = A_star(dists_road, dists_euclid, cities)
A.run('Aracajú', 'Curitiba')

A* algorithm - road between cities
start = Aracajú, end = Curitiba
Current best = Aracajú, f = 0
Current best = Belo Horizonte, f = 2399.3743703521613
Current best = Rio de Janeiro, f = 2531.477348201061
Current best = Maceió, f = 2556.1274519645985
Current best = Curitiba, f = 2582.0
Reached Curitiba
Path (in reverse order):
Curitiba
Belo Horizonte
Aracajú
